In [ ]:
import os.path as path
import raw_data_preprocessing as rdp
import warnings

warnings.filterwarnings('ignore')

googledrive_loc = '/datasets/googledrive'
data_folder_loc = 'Yi_UCI_research/GSR other works/2020 Summer_predict individual training/data/saved data/max_std/'
data_path = path.join(googledrive_loc, data_folder_loc)

import classification as clf
X, Y, orig_len, feature_names = clf.read_inputs(data_path,['input_x.array','input_y.array','original_len.int','feature_names.list'])
print('The input and output shape: {} and {}'.format(X.shape, Y.shape))

The input and output shape: (436, 14) and (436,)


In [ ]:
# clf.save_data(data_path, shuffle_list, 'shuffle.list')

In [ ]:
shuffle_list = clf.read_inputs(data_path, ['shuffle.list'])

In [ ]:
X = X[:orig_len]
result = []
acc = []
for i in range(111, 120):
    Y = shuffle_list[0][i]
    classify_test = clf.ClassificationProcessor(X,Y, orig_len, feature_names)
    final_accuracies, final_pred_y, final_prob_y, class0_acc, class1_acc, class2_acc, kappa = classify_test.final_model('Random Forest', [0, 1, 2,3,4,5,6,7,8])
    result.append(kappa)
    acc.append([class0_acc, class1_acc, class2_acc])
    print(i)

111
112
113
114
115
116


KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
import pandas as pd
pd.DataFrame(result).to_csv(data_path+'result_shuffle.csv')
pd.DataFrame(acc).to_csv(data_path+'acc_shuffle.csv')

In [ ]:
# classify_test = clf.ClassificationProcessor(X,Y, orig_len, feature_names)
# classify_test.get_stacking()

model selection and feature selection

In [ ]:
# classify_test.model_selection(14)

In [ ]:
# classify_test.exhausive_feature_selection('Nearest Neighbors')
# classify_test.exhausive_feature_selection('Random Forest')
# classify_test.feature_selection('Random Forest')

final model

In [ ]:
# final_accuracies, final_pred_y, final_prob_y = classify_test.final_model('Random Forest', [0, 1, 2,3,4,5,6,7,8])

In [ ]:
# final_accuracies, final_pred_y, final_prob_y = classify_test.final_model('Nearest Neighbors', [0, 1, 4, 6, 7, 9, 10, 12, 13])

shap

In [ ]:
# classify_test.shap('Random Forest',data_path)

In [ ]:
# shap_all, expected_shap = clf.read_inputs(data_path,['shap_values.data','shap_expected.data'])
# classify_test.shap_visualize(shap_all, expected_shap, final_accuracies, final_pred_y, False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6b8ead81-70bf-4d6b-bd50-8194f8f72b46' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>